In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_sorted2=pd.read_csv('/kaggle/input/cleanset/cleared_file.csv')

In [ ]:
#partitioning the data into sequential data and non sequential data
seq_cols=[cols for cols in df_sorted2.columns if 'txn' in cols]
X_seq=df_sorted2[seq_cols]
X_non_seq=df_sorted2.drop(columns=seq_cols)
X_non_seq=X_non_seq.drop(columns=['Target','Primary key'])
y=df_sorted2['Target']

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import numpy as np
X_seq_train, X_seq_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2, random_state=42)

# Standardize or normalize features for sequential data
scaler_seq = StandardScaler()
X_seq_train_scaled = scaler_seq.fit_transform(X_seq_train)
X_seq_test_scaled = scaler_seq.transform(X_seq_test)

# Reshape the input data for LSTM (assuming each sequence has 100 features)
sequence_length = X_seq_train_scaled.shape[1]
n_features = 1  # Assuming univariate time series data

X_seq_train_reshaped = X_seq_train_scaled.reshape((X_seq_train_scaled.shape[0], sequence_length * n_features))
X_seq_test_reshaped = X_seq_test_scaled.reshape((X_seq_test_scaled.shape[0], sequence_length * n_features))

# Apply SMOTE to the training set
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_seq_train_resampled, y_train_resampled = smote.fit_resample(X_seq_train_reshaped, y_train)

In [ ]:
X_non_seq_train, X_non_seq_test, y_train, y_test = train_test_split(X_non_seq, y, test_size=0.2, random_state=42)

# Standardize or normalize features for non-sequential data
scaler_non_seq = StandardScaler()
X_non_seq_train_scaled = scaler_non_seq.fit_transform(X_non_seq_train)
X_non_seq_test_scaled = scaler_non_seq.transform(X_non_seq_test)

# Apply SMOTE to the training set for handling class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_non_seq_train_resampled, y_train_resampled = smote.fit_resample(X_non_seq_train_scaled, y_train)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import ExponentialDecay
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_rate=0.9,
    decay_steps=1000,
    staircase=False  # Set to False for continuous decay
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np


In [ ]:
n_features_seq=1
model_seq = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(sequence_length, n_features_seq)),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(32, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])
#model_seq is a LSTM model to capture the sequential relations and patterns in the transactions
model_seq.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(),
                                                                         tf.keras.metrics.Recall()])

model_seq.summary()  # Print model summary

model_seq.fit(X_seq_train_resampled, y_train_resampled, epochs=15, batch_size=32, validation_split=0.2)

In [ ]:
model_non_seq = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=X_non_seq_train_scaled.shape[1]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
#model_non_seq to capture the patterns in non sequential data like demographic details, days of acc etc
model_non_seq.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
                          ])
model_non_seq.fit(X_non_seq_train_resampled, y_train_resampled,epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_seq_scaled=scaler.fit_transform(X_seq)
X_non_seq_scaled=scaler.fit_transform(X_non_seq)


In [ ]:
X_seq_reshaped = X_seq_scaled.reshape((X_seq_scaled.shape[0], sequence_length, n_features_seq))

In [ ]:
sq=model_seq.predict(X_seq_reshaped)


In [ ]:
nsq=model_non_seq.predict(X_non_seq_scaled)

In [ ]:
sq

In [ ]:
nsq

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
x1=df_sorted2.drop(columns=['Target','Primary key'])
y1=df_sorted2['Target']
x, y = smote.fit_resample(x1, y1)
# Assuming 'data' is your DataFrame, 'X' contains features, and 'y' is the target variable
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create an XGBoost classifier with adjusted hyperparameters
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    gamma=0,
    random_state=42
)

# Train the model
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Display the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Display additional metrics
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

#training a random forest classifier to extract less complex realation out of the dataset , by tuning hyperparameters we restrict overfitting

In [ ]:
df_fill=pd.read_csv('/kaggle/input/filldata/filldata.csv')

In [ ]:
df_fill.shape

In [ ]:
tree=model.predict_proba(x1)[:,1]

In [ ]:
sq=sq.ravel()
nsq=nsq.ravel()


In [ ]:
df_final=pd.DataFrame({'seq':sq})

In [ ]:
df_final['nonseq']=pd.DataFrame(nsq)

In [ ]:
df_final['tree']=pd.DataFrame(tree)
df_final['target']=pd.DataFrame(y)

In [ ]:
df_final

rather than depending only on deep models for the classification we are also taking weightage of ensemble methods here, XGBoost classifier. This comes in handy in case the deep model overfits the training dataset and captures noise in the data ,anyways we have taken proper methods  to prevent overfitting . And the models individually performing well with dataset gave away a motivation to include both.Basically if the  deep models are capturing too complex features including noise it may create a problem however the ensemble method does not capture such intricate(possibly noisy) patterns, thus overall decreasing the probability that comes just due to noise.

In [ ]:


#we obtained the weights using a multi-feature linear regression to calculate the optimum weights tested against the target value with gradient descent algorithm

a1=0.7 #sequential
a2=1.2 #non sequential
a3=2.5 #tree
df_final['preds']=(a1*df_final['seq']+a2*df_final['nonseq']+a3*df_final['tree'])/(a1+a2+a3)
df_final['predsf']=df_final['preds'].apply(lambda x: 1 if x > 0.5 else 0)
match = (df_final['predsf'] == df_final['target']).sum()
print(match) #checking the number of matching cells 